<a href="https://colab.research.google.com/github/DungTran206/Mistral-DAP/blob/main/mistral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Fix padding
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
def build_summary_prompt(text):
    return f"""
You are a summarization module in a multi-stage emotion analysis system.

Summarize the input text in 1 sentences ONLY, as short as possible.
Rewrite the input as a high-level description of the situation and emotions.
Do not preserve the original wording.
Preserve emotionally relevant information such as feelings, attitudes, and comparisons.
Do NOT analyze, explain, or label emotions.
Do NOT add new information.

Input text:
{text}

Output an extremely concise (around 30-40% of the original length), emotionally faithful summary:"""



In [ ]:
def summarize_text(text, max_new_tokens=80):
    prompt = build_summary_prompt(text)

    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.3,
            do_sample=False
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Lấy phần sau prompt
    return decoded.split("Output an extremely concise (around 30-40% of the original length), emotionally faithful summary:")[-1].strip()


In [ ]:
def batch_summarize(texts, batch_size=4):
    summaries = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        prompts = [build_summary_prompt(t) for t in batch]

        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=4096
        ).to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=60,
                temperature=0.2,
                do_sample=False
            )

        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)


        for d in decoded:
            summaries.append(
                d.split("Output an extremely concise (around 30-40% of the original length), emotionally faithful summary:")[-1].strip()
            )

    return summaries


In [ ]:
text = "Honestly, I’m beyond frustrated with how this project is going. I’ve been staring at the screen for hrs, and tbh, it feels like I’m running in circles. My teammate said he’d send the data by EOD, but ofc, he ghosted me again. It’s so draining when you put in 100% and get zero communication in return. I’m srsly starting to doubt if we can even meet the deadline. IDK, maybe I’m just overthinking, but right now, I’m just totally fed up with the lack of accountability. I need a break before I lose it."
summary = summarize_text(text)
print(summary)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Despite putting in maximum effort, the speaker is disheartened by the lack of progress on the project, exacerbated by their teammate's unreliability and poor communication, leading to frustration and doubt about meeting the deadline.


In [ ]:
print(len(summary.split(" ")) / len(text.split(" ")) * 100)

36.17021276595745
